In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from pathlib import Path

import spatialdata as sd
from loguru import logger

from multiplex_pipeline.core_quantification.controller import (
    QuantificationController,
)
from multiplex_pipeline.utils.utils import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA-1_Analysis\analysis_settings_BLCA1.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-1/BLCA-1_Final',
 'analysis_dir': 'C:/BLCA-1_Analysis',
 'log_dir': WindowsPath('C:/BLCA-1_Analysis/logs'),
 'detection_image': 'BLCA-1_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-1_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-1_Analysis/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-1_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instaseg',
  'model': 'fluorescence_nuclei_and_cells',
  'kwargs': {'pixel_size': 0.3,
   'resolve_cell_and_nucleus': True,
   'cleanup_fr

### Define the logger

In [3]:
log_file = settings['log_dir'] / f"cores_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [4]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-1_Analysis/cores/Core_000.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_001.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_002.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_003.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_004.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_005.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_006.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_007.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_008.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_009.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_010.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_011.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_012.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_013.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_014.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_015.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_016.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_017.

### Setup

In [5]:
controller = QuantificationController(
    mask_keys={'cell': 'instanseg_cell', 'nucleus': 'instanseg_nucleus'},
    table_name = 'instanseg_data',
    cytoplasm_components=('instanseg_cell', 'instanseg_nucleus'),
    cytoplasm_mask_name='cyto'
)

### Quantify

In [ ]:
for sd_path in path_list[6:]:

    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-09-24 13:10:22.297 | INFO     | __main__:<module>:4 - Processing Core_006.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mi

2025-09-24 13:10:31.973 | INFO     | multiplex_pipeline.core_quantification.controller:prepare_masks:46 - Cytoplasm mask 'cyto' created as instanseg_cell minus instanseg_nucleus.
2025-09-24 13:10:32.005 | INFO     | multiplex_pipeline.core_quantification.controller:run:73 - Prepared masks for quantification.
2025-09-24 13:10:32.005 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cell'
2025-09-24 13:11:23.381 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'nucleus'
2025-09-24 13:12:07.695 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cyto'
2025-09-24 13:12:57.881 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channel '53BP1' with mask 'cell'
2025-09-24 13:13:01.979 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channe

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)
version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


...

2025-09-24 14:14:46.256 | INFO     | multiplex_pipeline.core_quantification.controller:prepare_masks:46 - Cytoplasm mask 'cyto' created as instanseg_cell minus instanseg_nucleus.
2025-09-24 14:14:46.319 | INFO     | multiplex_pipeline.core_quantification.controller:run:73 - Prepared masks for quantification.
2025-09-24 14:14:46.320 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cell'
2025-09-24 14:15:36.061 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'nucleus'
2025-09-24 14:16:16.485 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cyto'
2025-09-24 14:16:57.109 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channel '53BP1' with mask 'cell'
2025-09-24 14:17:00.986 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channe

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)
version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-09-24 14:27:48.494 | SUCCESS  | multiplex_pipeline.core_quantification.controller:run:144 - Quantification complete. Table 'instanseg_data' written to C:\BLCA-1_Analysis\cores\Core_012.zarr
2025-09-24 14:27:48.604 | INFO     | __main__:<module>:4 - Processing Core_013.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mi

2025-09-24 14:27:58.034 | INFO     | multiplex_pipeline.core_quantification.controller:prepare_masks:46 - Cytoplasm mask 'cyto' created as instanseg_cell minus instanseg_nucleus.
2025-09-24 14:27:58.075 | INFO     | multiplex_pipeline.core_quantification.controller:run:73 - Prepared masks for quantification.
2025-09-24 14:27:58.075 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cell'
2025-09-24 14:28:33.345 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'nucleus'
2025-09-24 14:29:02.259 | INFO     | multiplex_pipeline.core_quantification.controller:run:85 - Quantifying morphology features for mask 'cyto'
2025-09-24 14:29:32.276 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channel '53BP1' with mask 'cell'
2025-09-24 14:29:35.225 | INFO     | multiplex_pipeline.core_quantification.controller:run:96 - Quantifying channe